## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib

# Import API key
from config_google import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Gatton,AU,-27.5500,152.2833,53.64,91,0,0.94,clear sky
1,1,Tevaitoa,PF,-16.7833,-151.5000,77.52,74,68,17.38,broken clouds
2,2,Kruisfontein,ZA,-34.0033,24.7314,47.88,78,9,4.03,clear sky
3,3,Bredasdorp,ZA,-34.5322,20.0403,51.44,93,2,4.21,clear sky
4,4,Urumqi,CN,43.8010,87.6005,59.95,77,75,6.71,broken clouds


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Tevaitoa,PF,-16.7833,-151.5000,77.52,74,68,17.38,broken clouds
5,5,Yellowknife,CA,62.4560,-114.3525,74.28,50,93,7.20,overcast clouds
6,6,Rikitea,PF,-23.1203,-134.9692,72.45,71,74,8.70,broken clouds
9,9,Katsuura,JP,35.1333,140.3000,79.11,86,11,8.88,few clouds
10,10,Ponta Do Sol,PT,32.6667,-17.1000,74.01,78,7,6.49,clear sky
11,11,Butaritari,KI,3.0707,172.7902,81.46,74,93,4.79,moderate rain
14,14,Bengkulu,ID,-3.8004,102.2655,74.59,92,24,5.17,few clouds
15,15,Ban Mo,TH,14.6154,100.7273,82.29,99,100,5.26,light rain
18,18,Avarua,CK,-21.2078,-159.7750,75.25,69,20,10.36,few clouds
19,19,Mayumba,GA,-3.4320,10.6554,71.85,87,36,7.85,scattered clouds


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0     365
City           365
Country        365
Lat            365
Lng            365
Max Temp       365
Humidity       365
Cloudiness     365
Wind Speed     365
Description    365
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df
clean_df.dropna()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Tevaitoa,PF,-16.7833,-151.5000,77.52,74,68,17.38,broken clouds
5,5,Yellowknife,CA,62.4560,-114.3525,74.28,50,93,7.20,overcast clouds
6,6,Rikitea,PF,-23.1203,-134.9692,72.45,71,74,8.70,broken clouds
9,9,Katsuura,JP,35.1333,140.3000,79.11,86,11,8.88,few clouds
10,10,Ponta Do Sol,PT,32.6667,-17.1000,74.01,78,7,6.49,clear sky
...,...,...,...,...,...,...,...,...,...,...
684,684,North Bend,US,43.4065,-124.2243,78.93,88,1,19.57,clear sky
687,687,Port Blair,IN,11.6667,92.7500,81.84,83,88,16.71,overcast clouds
688,688,Lagarto,BR,-10.9172,-37.6500,77.29,83,48,5.97,scattered clouds
690,690,Tucumcari,US,35.1717,-103.7250,94.95,26,40,12.66,scattered clouds


In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 8000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df
clean_hotel_df.dropna()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Tevaitoa,PF,77.52,broken clouds,-16.7833,-151.5000,Hôtel Raiatea Lodge
5,Yellowknife,CA,74.28,overcast clouds,62.4560,-114.3525,The Explorer Hotel
6,Rikitea,PF,72.45,broken clouds,-23.1203,-134.9692,People ThankYou
9,Katsuura,JP,79.11,few clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
10,Ponta Do Sol,PT,74.01,clear sky,32.6667,-17.1000,Hotel do Campo
...,...,...,...,...,...,...,...
684,North Bend,US,78.93,clear sky,43.4065,-124.2243,Best Western Holiday Hotel
687,Port Blair,IN,81.84,overcast clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
688,Lagarto,BR,77.29,scattered clouds,-10.9172,-37.6500,Pousada Cota
690,Tucumcari,US,94.95,scattered clouds,35.1717,-103.7250,"Holiday Inn Express & Suites Tucumcari, an IHG..."


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv" 

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))